# Final Project
### Daniel Mehta

---

## Introduction and Problem Statement

**Business Problem**  
Exchange-Traded Funds (ETFs) are popular investment products, but creating them manually requires extensive research and portfolio balancing.  
I propose an **AI-driven ETF grouping system** that automatically clusters stocks in the S&P 500 into ETF-style groups based on investment-relevant characteristics such as volatility, return, momentum, and liquidity.

These clusters could be used by financial institutions or fintech platforms to quickly generate investment ideas for different risk profiles and objectives. For example, aggressive growth, stable income, or balanced diversification.

**Note on Feature Selection**  
My original concept included sector classification as a feature, but the available dataset does not contain sector information.  
To maintain project simplicity and avoid external data merging, this implementation will focus solely on numerical, market-derived features.

**Goal**  
Build a stock clustering pipeline that:  
1. Processes historical price data for S&P 500 companies.  
2. Creates numerical feature vectors for each stock.  
3. Applies and compares **K-Means Clustering** (covered in course) and **Hierarchical Clustering** (new) to group the stocks.  
4. Evaluates clusters using quantitative metrics and visualizations.  
5. Recommends the optimal approach for deployment in an ETF grouping tool.

---

## Imports

In [8]:
import pandas as pd

---

### Load Dataset / Cleaning

In [12]:
file_path = "all_stocks_5yr.csv"
df = pd.read_csv(file_path)

In [16]:
df.head(),df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    619040 non-null  object 
 1   open    619029 non-null  float64
 2   high    619032 non-null  float64
 3   low     619032 non-null  float64
 4   close   619040 non-null  float64
 5   volume  619040 non-null  int64  
 6   Name    619040 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 33.1+ MB


(         date   open   high    low  close    volume Name
 0  2013-02-08  15.07  15.12  14.63  14.75   8407500  AAL
 1  2013-02-11  14.89  15.01  14.26  14.46   8882000  AAL
 2  2013-02-12  14.45  14.51  14.10  14.27   8126000  AAL
 3  2013-02-13  14.30  14.94  14.25  14.66  10259500  AAL
 4  2013-02-14  14.94  14.96  13.16  13.99  31879900  AAL,
 None)

In [18]:
# Converting date to datetime
df['date'] =pd.to_datetime(df['date'])

In [20]:
# check for missing values
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 date       0
open      11
high       8
low        8
close      0
volume     0
Name       0
dtype: int64


In [22]:
# Drop rows with any missing open, high, low, close values
df = df.dropna(subset=['open', 'high', 'low', 'close'])

In [24]:
#Checking correct data types
numeric_cols = ['open', 'high', 'low', 'close', 'volume']
df[numeric_cols] = df[numeric_cols].astype(float)

In [26]:
# Removeing any duplicates
df = df.drop_duplicates(subset=['date', 'Name'])

In [30]:
# Sort by Name then date
df =df.sort_values(by=['Name','date']).reset_index(drop=True)

In [32]:
print(f"Cleaned dataset shape: {df.shape}")
df.head()

Cleaned dataset shape: (619029, 7)


,date,open,high,low,close,volume,Name
0,2013-02-08,45.07,45.35,45.00,45.08,1824755.0,A
1,2013-02-11,45.17,45.18,44.45,44.60,2915405.0,A
2,2013-02-12,44.81,44.95,44.50,44.62,2373731.0,A
3,2013-02-13,44.81,45.24,44.68,44.75,2052338.0,A
4,2013-02-14,44.72,44.78,44.36,44.58,3826245.0,A
